<a href="https://colab.research.google.com/github/lweislo/cycling-project/blob/master/OneDayRaceGrouping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import files

In [0]:
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}"'.format(
        name=fn, length=len(uploaded[fn])))


In [0]:
for item in uploaded:
    df = pd.read_csv(item, encoding='UTF-8')


In [0]:
# get rid of the DNFs
df1 = df[~df['Rank'].isin(['DNF','DNS','OTL','NQ','DSQ'])]
df1['Rank'].unique()

In [0]:
# Turn the ranking into an integer
df1['Rank'] = pd.to_numeric(df1['Rank'])
df1.dtypes

In [0]:
# Most wins
dfw = df1.loc[df1['Rank'] ==1]
wins = dfw.groupby('Rider', as_index=True)['Rank'].count()
wins.sort_values(ascending=False)

In [0]:
# Filter only for podium finishes
dfp = df1.loc[df1['Rank']<4]
dfpg = dfp.groupby(['Rider','Rank'], as_index=False)['Year'].count()
# Generating top 10 a medal table
medal_table = dfpg.pivot(index='Rider', columns='Rank')['Year']
medal_table.reset_index(inplace=True)
mt = medal_table.sort_values(by=[1,2,3], na_position="last",ascending=False)[:10]
mt

In [0]:
# Get median ranking for riders who've been in top 20
dfm = df1.groupby(['Rider'], as_index=False)['Rank'].median()
dfm.dropna(how='all',inplace=True)
dfm = dfm.loc[dfm['Rank']<21]
dfm = dfm.sort_values(by='Rank', ascending=True).head(20)
dfm.rename(columns={'Rank':'Median Rank'}, inplace=True)
dfm.head(2)

In [0]:
# Get the top 10 most raced in the dataset
dfg = df1.groupby(['Rider'], as_index=False)['Rank'].count()
dfg.dropna(how='all',inplace=True)
dfg = dfg.sort_values(by=['Rank'], ascending=False)
dfg.rename(columns={'Rank':'Finishes'}, inplace=True)
dfg.head(2)

In [0]:
# Limit the group to riders who have finished at least three times
dfgf = dfg.loc[dfg['Finishes']>2]
dfgf.head(3)

In [0]:
# Merge the two on rider to create a most successful rider ranking
df2 = dfgf.merge(dfm, how='inner', on='Rider')
df2.sort_values(by=('Finishes'), ascending=False).sort_values(by=(['Median Rank']), ascending=True).head(10)

In [0]:
# List of all winners in dataset
dfw.sort_values('Year',ascending=False)

In [0]:
# output = df1.to_excel('E3_results_noDNF.xlsx', index=False)
# files.download('E3_results_noDNF.xlsx')

In [0]:
# Most top 10s without win
top10 = np.arange(2,11)
df3 = df1.loc[df1['Rank'].isin(top10)]
df3g = df3.groupby('Rider', as_index=False)['Rank'].count()
unluckiest = df3g.sort_values('Rank',ascending=False)

In [0]:
# Top 10 unluckiest riders - top 10s without wins
unluckiest[:10]

In [0]:
# Youngest winner
dfw.sort_values(by='Age')[:1]

In [0]:
# Oldest winner
dfw.sort_values(by='Age',ascending=False)[:1]

In [0]:
# Fastest edition
dfw.sort_values(by='Speed',ascending=False)[:1]

In [0]:
# Slowest edition
dfw.sort_values(by='Speed',ascending=True)[:1]

In [0]:
# Mean age/speed/time
dfw.describe()